## Połączenie z bazą danych przy pomocy psycopg2 i pandas oraz dołączenie modułu main

In [1]:
import main

import numpy as np
import pickle
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS.

***1. Znajdź liczbę wszystkich filmów o tej samej długości.***

In [2]:
sql = '''SELECT
             length, COUNT(title) number_of_films
         FROM
             film
         GROUP BY
             length;'''

pd.read_sql(sql, con=main.connection)

,length,number_of_films
0,129,7
1,106,6
2,120,9
3,171,8
4,138,6
...,...,...
135,159,6
136,72,4
137,177,6
138,141,7


***2. Znajdź wszystkich klientów mieszkających w tym samym mieście.***

In [3]:
sql = f'''SELECT
              city.city, STRING_AGG(CONCAT(customer.first_name, ' ', customer.last_name), ', '
                                      ORDER BY customer.last_name) AS customers, COUNT(customer.first_name) number_of_customers
          FROM
              city
          INNER JOIN address ON city.city_id=address.city_id
          INNER JOIN customer ON address.address_id=customer.address_id
          GROUP BY
              city.city;'''

pd.read_sql(sql, con=main.connection)

,city,customers,number_of_customers
0,A Corua (La Corua),Julie Sanchez,1
1,Abha,Peggy Myers,1
2,Abu Dhabi,Tom Milner,1
3,Acua,Glen Talbert,1
4,Adana,Larry Thrasher,1
...,...,...,...
592,Zaria,Constance Reid,1
593,Zeleznogorsk,Jack Foust,1
594,Zhezqazghan,Byron Box,1
595,Zhoushan,Guy Brownlee,1


***3. Oblicz średni koszt wypożyczenia wszystkich filmów.***

In [5]:
sql = '''SELECT
             AVG(amount) average_payment
         FROM
             payment;'''

pd.read_sql(sql, con=main.connection)

,average_payment
0,4.200606


***4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.***

In [6]:
sql = '''SELECT
             category.name, COUNT(film.title) number_of_films
         FROM
             category
         INNER JOIN film_category ON category.category_id=film_category.category_id
         INNER JOIN film ON film.film_id=film_category.film_id
         GROUP BY
             category.name
         ORDER BY
             category.name;'''

pd.read_sql(sql, con=main.connection)

,name,number_of_films
0,Action,64
1,Animation,66
2,Children,60
3,Classics,57
4,Comedy,58
5,Documentary,68
6,Drama,62
7,Family,69
8,Foreign,73
9,Games,61


***5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.***

In [7]:
sql = '''SELECT
             country.country, STRING_AGG(CONCAT(customer.first_name, ' ', customer.last_name), ', '
                                      ORDER BY customer.last_name) AS customers, COUNT(customer.first_name) number_of_customers
         FROM
             customer
         INNER JOIN address ON customer.address_id=address.address_id
         INNER JOIN city ON address.city_id=city.city_id
         INNER JOIN country ON country.country_id=city.country_id
         GROUP BY
             country.country
         ORDER BY
             country.country;'''

pd.read_sql(sql, con=main.connection)

,country,customers,number_of_customers
0,Afghanistan,Vera Mccoy,1
1,Algeria,"June Carroll, Mario Cheatham, Judy Gray",3
2,American Samoa,Anthony Schwab,1
3,Angola,"Martin Bales, Claude Herzog",2
4,Anguilla,Bobby Boudreau,1
...,...,...,...
103,Vietnam,"Beth Franklin, Harvey Guajardo, Patrick Newsom...",6
104,"Virgin Islands, U.S.",Nathan Runyon,1
105,Yemen,"Gordon Allard, Ella Oliver, William Satterfiel...",4
106,Yugoslavia,"Maria Miller, Max Pitt",2


***6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.***

In [8]:
sql = '''SELECT address.address, store.count number_of_customers
         FROM (
         SELECT
             store.address_id, COUNT(customer.customer_id)
         FROM
             customer
         INNER JOIN store ON customer.store_id=store.store_id
         GROUP BY
             store.store_id
         HAVING COUNT(customer.customer_id) BETWEEN 101
         AND 299) AS store
         INNER JOIN address ON store.address_id=address.address_id;'''

pd.read_sql(sql, con=main.connection)

,address,number_of_customers
0,28 MySQL Boulevard,273


***7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.***

In [9]:
sql = '''SELECT
             customer.first_name, customer.last_name, SUM(film.length) total_film_length
         FROM
             customer
         INNER JOIN rental ON customer.customer_id=rental.customer_id
         INNER JOIN inventory ON rental.inventory_id=inventory.inventory_id
         INNER JOIN film ON inventory.film_id=film.film_id
         GROUP BY
             customer.customer_id
         HAVING
             SUM(film.length) > 200 * 60;'''

pd.read_sql(sql, con=main.connection)

,first_name,last_name,total_film_length


Długość filmu jest podawana w minutach.
Nikt nie oglądał filmów ponad 200 godzin.

***8. Oblicz średnią wartość wypożyczenia filmu.***

In [10]:
sql = '''SELECT
             AVG(amount)
         FROM
             payment;'''

pd.read_sql(sql, con=main.connection)

,avg
0,4.200606


Zadanie tożsame z zadaniem 3.

***9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.***

In [11]:
sql = f'''SELECT
              category.name, AVG(film.length) average_film_length
          FROM
              category
          INNER JOIN film_category ON category.category_id=film_category.category_id
          INNER JOIN film ON film_category.film_id=film.film_id
          GROUP BY
              category.name;'''

pd.read_sql(sql, con=main.connection)

,name,average_film_length
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


Długość filmu jest podawana w minutach.

***10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.***

In [12]:
sql = f'''SELECT
              name, STRING_AGG(title, ', ') AS titles, title_length
          FROM (
              SELECT
                  f.name, f.title, c.max title_length
              FROM (
                  SELECT
                      category.name, film.title
                  FROM
                      category
                  INNER JOIN film_category ON category.category_id=film_category.category_id
                  INNER JOIN film ON film_category.film_id=film.film_id) AS f
              INNER JOIN (
                  SELECT
                      category.name, MAX(LENGTH(film.title))
                  FROM
                      category
                  INNER JOIN film_category ON category.category_id=film_category.category_id
                  INNER JOIN film ON film_category.film_id=film.film_id
                  GROUP BY
                      category.name) AS c ON f.name=c.name AND LENGTH(f.title)=c.max) AS _
          GROUP BY
              name, title_length;'''

pd.read_sql(sql, con=main.connection)

,name,titles,title_length
0,Action,Entrapment Satisfaction,23
1,Animation,Telemark Heartbreakers,22
2,Children,"Heartbreakers Bright, Microcosmos Paradise, Sw...",20
3,Classics,Extraordinary Conquerer,23
4,Comedy,Trainspotting Strangers,23
5,Documentary,"Intolerable Intentions, Deliverance Mulholland",22
6,Drama,Goldfinger Sensibility,22
7,Family,Resurrection Silverado,22
8,Foreign,"Reservoir Adaptation, Impossible Prejudice, Ba...",20
9,Games,"Creatures Shakespeare, Bulworth Commandments",21


***11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.***

In [13]:
sql = f'''SELECT
              name, STRING_AGG(title, ', ') AS titles, length
          FROM (
              SELECT
                  f.name, f.title, f.length
              FROM (
                  SELECT
                      category.name, film.title, film.length
                  FROM
                      category
                  INNER JOIN film_category ON category.category_id=film_category.category_id
                  INNER JOIN film ON film_category.film_id=film.film_id) AS f
              INNER JOIN (
                  SELECT
                      category.name, MAX(film.length)
                  FROM
                      category
                  INNER JOIN film_category ON category.category_id=film_category.category_id
                  INNER JOIN film ON film_category.film_id=film.film_id
                  GROUP BY
                      category.name) AS c ON f.name=c.name AND f.length=c.max) AS _
          GROUP BY
              name, length;'''

pd.read_sql(sql, con=main.connection)

,name,titles,length
0,Action,"Darn Forrester, Worst Banger",185
1,Animation,"Pond Seattle, Gangs Pride",185
2,Children,"Fury Murder, Wrong Behavior",178
3,Classics,Conspiracy Spirit,184
4,Comedy,Control Anthem,185
5,Documentary,"Young Language, Wife Turn",183
6,Drama,Jacket Frisco,181
7,Family,King Evolution,184
8,Foreign,"Sorority Queen, Crystal Breaking",184
9,Games,Chicago North,185


Długość filmu jest podawana w minutach.
Porównanie wyników z zadaniem 10. jest niemożliwe - długość filmu nie zależy od długości tytułu.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [19]:
main.film_in_category(15)

,title,languge,category
0,Aladdin Calendar,English,Sports
1,Anonymous Human,English,Sports
2,Artist Coldblooded,English,Sports
3,Bubble Grosse,English,Sports
4,California Birds,English,Sports
...,...,...,...
69,Tights Dawn,English,Sports
70,Tourist Pelican,English,Sports
71,Trading Pinocchio,English,Sports
72,Tuxedo Mile,English,Sports


In [20]:
main.number_films_in_category(15)

,category,count
0,Sports,74


Liczba filmów w kategorii Sports się zgadza.

In [23]:
main.number_film_by_length(min_length=60, max_length=80.5)

,length,count
0,80,10
1,61,10
2,67,8
3,77,6
4,66,2
5,73,12
6,62,6
7,65,7
8,76,7
9,69,6


In [24]:
main.client_from_city("Aurora")

,city,first_name,last_name
0,Aurora,Clinton,Buford
1,Aurora,Scott,Shelley


In [27]:
main.avg_amount_by_length(80)

,length,avg
0,80,5.48359


In [30]:
main.client_by_sum_length(4308)

,first_name,last_name,sum
0,Sue,Peters,4319
1,Louis,Leone,4333
2,Angela,Hernandez,4362
3,Janice,Ward,4366
4,Daisy,Bates,4369
5,Tim,Cary,4476
6,Diane,Collins,4514
7,Elizabeth,Brown,4531
8,Justin,Ngo,4550
9,Sherry,Marshall,4582


In [32]:
main.category_statistic_length("Sports")

,category,avg,sum,min,max
0,Sports,128.202703,9487,47,184
